<a href="https://colab.research.google.com/github/liamglennon/College/blob/main/ElectroMagSim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coilgun Simulation Notebook

This interactive notebook simulates the dynamics of a coilgun projectile using a simplified electromagnetic model.

### Features
- Adjustable parameters such as initial voltage, coil turns, efficiency, capacitance, and resistance
- Real-time visualizations of projectile dynamics:
  - Position and velocity over time
  - Magnetic field strength and acceleration
  - Magnetic field versus projectile position
- A basic look at how energy transfer and electromagnetic force influence projectile motion

Use the sliders below to explore how different parameters impact the coilgun's performance.



In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Button, VBox, HBox, Output
import pandas as pd
import io
from IPython.display import FileLink, display
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# Magnetic field computation
# -----------------------------
def magnetic_field(distance, turns, current):
    mu_0 = 4 * np.pi * 1e-7
    return mu_0 * turns * current / np.where(distance != 0, distance, 1e-20)

# -----------------------------
# Main simulation function
# -----------------------------
def simulate_coil_gun(
    dt=0.001,
    total_time=0.1,
    initial_distance=0.1,
    initial_velocity=0,
    resistance=2,
    capacitance=8000e-6,
    voltage=20.2,
    efficiency=0.001,
    turns=103,
    mass=0.006
):
    times = np.arange(0, total_time, dt)
    positions, velocities, accelerations, fields = [], [], [], []

    position = initial_distance
    velocity = initial_velocity
    energized = True
    charge = capacitance * voltage
    mu_0 = 4 * np.pi * 1e-7

    for time in times:
        current = charge / capacitance
        if position < 0.01:
            energized = False

        field = magnetic_field(position, turns, current) if energized else 0
        force = mu_0 * turns**2 * current**2 / (2 * position**2 if position != 0 else 1e-20) if energized else 0
        acceleration = (force / mass) * efficiency

        velocity += acceleration * dt
        position += velocity * dt

        positions.append(position)
        velocities.append(velocity)
        accelerations.append(acceleration)
        fields.append(field)

        charge -= current * resistance * dt
        if position <= 0 or charge < 0:
            break

    return times[:len(positions)], positions, velocities, accelerations, fields

# -----------------------------
# CSV Export Utility
# -----------------------------
def export_to_csv(times, positions, velocities, accelerations, fields):
    df = pd.DataFrame({
        'Time (s)': times,
        'Position (m)': positions,
        'Velocity (m/s)': velocities,
        'Acceleration (m/s²)': accelerations,
        'Magnetic Field (T)': fields
    })
    buffer = io.StringIO()
    df.to_csv(buffer, index=False)
    buffer.seek(0)
    filename = 'coilgun_simulation.csv'
    with open(filename, 'w') as f:
        f.write(buffer.read())
    display(FileLink(filename))

# -----------------------------
# Plotting + CSV Export Combined
# -----------------------------
def run_sim_and_plot(
    voltage,
    turns,
    efficiency,
    initial_distance,
    capacitance,
    resistance,
    total_time,
    dt,
    mass
):
    times, positions, velocities, accelerations, fields = simulate_coil_gun(
        voltage=voltage,
        turns=turns,
        efficiency=efficiency,
        initial_distance=initial_distance,
        capacitance=capacitance,
        resistance=resistance,
        total_time=total_time,
        dt=dt,
        mass=mass
    )

    # Plotting results
    plt.figure(figsize=(12, 12))

    plt.subplot(4, 1, 1)
    plt.plot(times, positions, label='Position (m)')
    plt.plot(times, velocities, label='Velocity (m/s)')
    plt.title('Projectile Dynamics Over Time')
    plt.xlabel('Time (s)')
    plt.legend()

    plt.subplot(4, 1, 2)
    plt.plot(times, fields, label='Magnetic Field Strength (T)')
    plt.title('Magnetic Field Strength Along Projectile Path')
    plt.ylabel('Magnetic Field (T)')
    plt.xlabel('Time (s)')
    plt.legend()

    plt.subplot(4, 1, 3)
    plt.plot(times, accelerations, label='Acceleration (m/s²)')
    plt.title('Acceleration of Projectile')
    plt.ylabel('Acceleration (m/s²)')
    plt.xlabel('Time (s)')
    plt.legend()

    plt.subplot(4, 1, 4)
    plt.plot(positions, fields, label='Magnetic Field vs. Position')
    plt.title('Magnetic Field vs. Projectile Position')
    plt.xlabel('Position (m)')
    plt.ylabel('Magnetic Field (T)')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Export to CSV
    export_to_csv(times, positions, velocities, accelerations, fields)

# -----------------------------
# Interactive UI with sliders
# -----------------------------
interact(
    run_sim_and_plot,
    voltage=FloatSlider(value=20.0, min=5.0, max=60.0, step=0.5, description='Voltage (V)'),
    turns=IntSlider(value=100, min=10, max=300, step=10, description='Coil Turns'),
    efficiency=FloatSlider(value=0.001, min=0.0001, max=0.01, step=0.0001, readout_format='.4f', description='Efficiency'),
    initial_distance=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Init Dist (m)'),
    capacitance=FloatSlider(value=8000e-6, min=100e-6, max=20000e-6, step=100e-6, readout_format='.1e', description='Capacitance (F)'),
    resistance=FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Resistance (Ω)'),
    total_time=FloatSlider(value=0.1, min=0.02, max=0.5, step=0.01, description='Total Time (s)'),
    dt=FloatSlider(value=0.001, min=0.0001, max=0.01, step=0.0001, readout_format='.4f', description='Time Step (s)'),
    mass=FloatSlider(value=0.006, min=0.001, max=0.05, step=0.001, description='Mass (kg)')
)







interactive(children=(FloatSlider(value=20.0, description='Voltage (V)', max=60.0, min=5.0, step=0.5), IntSlid…

<function __main__.run_sim_and_plot(voltage, turns, efficiency, initial_distance, capacitance, resistance, total_time, dt, mass)>